In [ ]:
# !pip install requests
# !pip install html5lib
# !pip install bs4

In [1]:
import requests
from bs4 import BeautifulSoup
from csv import writer

In [ ]:
def PerfumeAgent(soup):
  perfumerAgents = soup.find_all('div',class_ = "bnflexfluid1")
  perfumer = list()
  supplier =  list()
  packaging = list()
  originalBrand = list()
  createdFor = list()

  for j in perfumerAgents[1:]:
    agentsList=  j.text.split()
    if(agentsList[-1] == 'Perfumer'):
      agent = j.find('a').text
      perfumer.append(agent)
    elif(agentsList[-1] == 'Supplier'): 
      agent = j.find('a').text
      supplier.append(agent)
    elif(agentsList[-1] == 'for'): 
      agent = j.find('a').text
      createdFor.append(agent)
    elif(agentsList[-1] == 'House'): 
      agent = j.find('a').text
      originalBrand.append(agent)
    else:
      agent = j.find('a').text
      packaging.append(agent)
  
  Perfumer = " + ".join(perfumer)
  Supplier = " + ".join(supplier)
  Packaging = " + ".join(packaging)
  CreatedFor = " + ".join(createdFor)
  OriginalBrand= " + ".join(originalBrand)
  
  return Perfumer, Supplier, Packaging , CreatedFor, OriginalBrand

In [ ]:
def Availability(soup):
  avail = soup.find('h1',class_ = 'font-heavy').find('span', {'class',"",'style', "background:black;padding:8px;border-radius:5px;font-size:small;text-tranform:uppercase;" })
  if(avail == None):
    availability = "INPRODUCTION"
  else:
    availability = avail.text
  return availability

In [ ]:
def helper(notes):
  array = list()
  for i in notes:
   if(len(i)!=0):
    array.append(i)
   else:
     pass
  array= " + ".join(array)
  return array

In [ ]:
def FraganceNotes(soup):
  topNotes = [] 
  heartNotes = []  
  baseNotes= [] 
  generalNotes = []
  fraganceNotes = soup.find_all('ol', class_ = "fragrancenotes")
  if(len(fraganceNotes) == 0):
    return topNotes, heartNotes, baseNotes, generalNotes

  notesCheck = fraganceNotes[0].find('li').find('h3')
  fraganceNotes = fraganceNotes[0].find_all('ul')

  if(notesCheck != None):
    top = fraganceNotes[0].get_text().split('\n')
    topNotes = helper(top)
    heart = fraganceNotes[1].get_text().split('\n')
    heartNotes = helper(heart)
    base = fraganceNotes[2].get_text().split('\n')
    baseNotes = helper(base)

  else:
    general = fraganceNotes[0].get_text().split('\n')
    generalNotes = helper(general)

  return topNotes, heartNotes, baseNotes, generalNotes

In [ ]:
def Gender(soup):
  F = soup.find_all('i', class_ = "genderF fas fa-lg fa-venus")
  M = soup.find_all('i', class_ = "genderM fas fa-lg fa-mars")
  U = soup.find_all('i', class_ = "genderS fas fa-lg fa-venus-mars")
  if(len(F) != 0):
    return "Feminine"
  if (len(M) !=0):
    return "Masculine"
  if (len(U) != 0):
    return "Shared/Unisex"

In [ ]:
def rating(soup):
  Nostar = soup.find_all('i', class_ = "fal fa-star")
  starsF = soup.find_all('i', class_ = "fas fa-star")
  starsH = soup.find_all('i', class_ = "fas fa-star-half-alt")
  if(len(Nostar)==1 & len(starsF)==0):
    return 0
  if(len(starsH)!=0):
    full = len(starsF)+0.5
  else:
     full = len(starsF)
  return full

In [ ]:
name = ''
gender = ''
year_of_launch = ''
house = ''
avg_rating = 0
availability = ''
perfumer =''
packaging = ''
parent_company = ''
parent_company_at_launch = ''
supplier = ''
votes = 0
topNotes = ''
heartNotes = ''
baseNotes = ''
generalNotes = ''
count =1
page = 1

with open('perfume.csv', 'w', encoding='utf8', newline='') as f:
  thewriter = writer(f)
  header = ['name','year_of_launch','gender','availability','votes','avg_rating','house','perfumer','packaging','parent_company','supplier','parent_company_at_launch','top_notes','heart_notes','base_notes','general_notes']
  thewriter.writerow(header)

  while page != 35:
    url = f"https://basenotes.com/fragrances/page-{page}?launch=2020-2029"
    pageUrl = requests.get(url)
    Soup = BeautifulSoup(pageUrl.content, 'html.parser')
    lists = Soup.find_all('div', class_="bncard card6")
    print("Page ",page)
    for i in lists:

      print("Product ",count)
      link = f'https://basenotes.com{i.find("a")["href"]}'
      # print(link)
      eachPage = requests.get(link)
      soup = BeautifulSoup(eachPage.content, 'html.parser')
      heading = soup.find('h1',class_ = 'font-heavy')
      if(heading == None):
        print("product ",count," html page couldn't load")
      else:
        year =  i.find('p').text.split()
        year_of_launch = year[-1].replace('(',"").replace(')',"")
        name  = heading.find('span',{'itemprop':"Name"}).text
        house = heading.find('span',{'itemprop':"name"}).text
        avg_rating = rating(soup)
        reviewsCheck = soup.find('i', class_ = "fas fa-comment-exclamation")
        if(reviewsCheck == None):
          votes =0
        else:
          reviews = soup.find('div', class_ = "bntopdeets").find_all('span')
          votes = reviews[-1].text.split()[0]
        gender = Gender(soup)
        availability = Availability(soup)
        perfumer, supplier,packaging, parent_company_at_launch, parent_company = PerfumeAgent(soup)
        topNotes, heartNotes, baseNotes, generalNotes = FraganceNotes(soup)
        
        info = [name,year_of_launch,gender,availability,votes,avg_rating,house,perfumer,packaging,parent_company,supplier,parent_company_at_launch,topNotes,heartNotes,baseNotes,generalNotes]
        thewriter.writerow(info)

      count = count+1
      
    page = page + 1

print("-------------------------------------Data Collected--------------------------------------------------------")